In [1]:
import pandas as pd
import pickle
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from statsmodels.iolib.smpickle import load_pickle

dependent_vars = ['Depressed', 'Anxious', 'Received_Free_Food', 'Food_Insecurity', 'Difficulty_with_Expenses', 'Rent_Confidence']
dependent_vars = ['Received_Free_Food', 'Food_Insecurity', 'Enrolled_in_SNAP']

In [3]:

results = []
for var in dependent_vars:
    models = []
    for kids in ["None", "1", "2", "3", "4"]:
        models.append(load_pickle(f"models\{var}_fe_model_{kids}_kids.p"))
    output_table = pd.read_html(
        summary_col([*models],
                    stars=True,
                    regressor_order=["Treat"],
                    drop_omitted=True,
                    info_dict={
            "N": lambda x: (str(int(x.nobs))), "Fixed Effects": lambda x: "Week/Group"})
        .as_html(), header=0, index_col=0)[0]
    output_table.columns = ['Children Generally', 'One Child', 'Two Children', 'Three Children', 'Four Children']
    output_table.index = 'Treat', '', 'R-squared', 'R-squared Adj.', 'N', 'Fixed Effects'
    results.append(output_table)

In [4]:
new_index = []
for ind, var in enumerate(dependent_vars):
    new_index.append(var)
    if ind+1 % 2 != 0:
        new_index.append("")

In [5]:
all_results = pd.concat([result.iloc[:2, :] for result in results])
all_results.index = new_index
print(pd.concat([all_results, results[0].iloc[-2:, :]]).to_csv().replace("_", " "))

,Children Generally,One Child,Two Children,Three Children,Four Children
Received Free Food,-0.0181***,-0.0141***,-0.0168***,-0.0218**,-0.0485***
,(0.0033),(0.0042),(0.0053),(0.0095),(0.0169)
Food Insecurity,-0.0256***,-0.0206***,-0.0264***,-0.0375***,-0.0395**
,(0.0037),(0.0049),(0.0058),(0.0090),(0.0180)
Enrolled in SNAP,0.0102**,0.0126**,0.0047,0.0196,0.0080
,(0.0046),(0.0058),(0.0073),(0.0134),(0.0222)
N,842783,713637,679113,628867,608418
Fixed Effects,Week/Group,Week/Group,Week/Group,Week/Group,Week/Group



In [6]:

models = []
for var in dependent_vars:
    models.append(load_pickle(f"models\{var}_event_study_None_kids.p"))


In [7]:
output_table = pd.read_html(
    summary_col([*models],
                stars=True,
                info_dict={
        "N": lambda x: (str(int(x.nobs)))})
    .as_html(), header=0, index_col=0)[0]
# output_table.columns = ['Children Generally', 'One Child',
#                         'Two Children', 'Three Children', 'Four Children']
# output_table.index = 'Treat', '', 'R-squared', 'R-squared Adj.', 'N', 'Fixed Effects'
# results.append(output_table)

In [8]:
output_table = output_table.reset_index()

In [9]:
print(output_table[output_table['index'].str.startswith("C(Week)") & output_table['index'].str.endswith(":Eligible")].rename(columns={"index": "Coefficient"}).to_csv(index=False).replace("_", " "))

Coefficient,Received Free Food,Food Insecurity,Enrolled in SNAP
C(Week)[T.27]:Eligible,-0.0204**,-0.0231**,-0.0165
C(Week)[T.28]:Eligible,-0.0113,-0.0175,-0.0122
C(Week)[T.29]:Eligible,-0.0194**,-0.0101,0.0181
C(Week)[T.30]:Eligible,-0.0076,-0.0007,0.0077
C(Week)[T.31]:Eligible,-0.0268***,-0.0072,0.0172
C(Week)[T.32]:Eligible,-0.0289***,-0.0008,0.0032
C(Week)[T.33]:Eligible,-0.0169*,0.0087,0.0110
C(Week)[T.34]:Eligible,-0.0425***,-0.0358***,0.0299**
C(Week)[T.35]:Eligible,-0.0582***,-0.0416***,0.0040
C(Week)[T.36]:Eligible,-0.0522***,-0.0261**,0.0069
C(Week)[T.37]:Eligible,-0.0536***,-0.0347***,0.0148
C(Week)[T.38]:Eligible,-0.0539***,-0.0324***,0.0032
C(Week)[T.39]:Eligible,-0.0382***,-0.0375***,0.0117
C(Week)[T.40]:Eligible,-0.0382***,-0.0261**,0.0137
C(Week)[T.41]:Eligible,-0.0508***,-0.0184*,0.0068
C(Week)[T.42]:Eligible,-0.0496***,-0.0055,0.0047
C(Week)[T.43]:Eligible,-0.0606***,-0.0153,-0.0094
C(Week)[T.44]:Eligible,-0.0563***,-0.0117,-0.0104
C(Week)[T.45]:Eligible,-0.0460***,0.0

In [10]:
output_table

,index,Received_Free_Food,Food_Insecurity,Enrolled_in_SNAP
0,Intercept,0.0595***,0.0879***,0.0874***
1,NaN,(0.0028),(0.0038),(0.0036)
2,C(Week)[T.27],-0.0021,-0.0056,-0.0023
3,NaN,(0.0038),(0.0054),(0.0050)
4,C(Week)[T.28],-0.0059,-0.0192***,-0.0039
...,...,...,...,...
78,C(Week)[T.45]:Eligible,-0.0460***,0.0035,-0.0122
79,NaN,(0.0103),(0.0119),(0.0128)
80,R-squared,0.0154,0.0088,0.0372
81,R-squared Adj.,0.0153,0.0088,0.0372
